In [3]:
import pandas as pd
import numpy as np
import math
import time

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [4]:
vic_data = pd.read_csv('../kennedy/victoria.csv')

In [5]:
# Constant Variables 
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
EFFICIENCY = 0.9
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

CHARGE = -1
DISCHARGE = 1

In [16]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [17]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    df['decision1'] = 0
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    df = df.drop([0], axis=0) 
    
    return df

In [18]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'decision1'] = DISCHARGE
                
        else:
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)

            if (df.at[i,PRICE] <= ser.quantile(CHARGING_PERCENTILE)):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= ser.quantile(DISCHARGING_PERCENTILE)):
                df.at[i,'decision1'] = DISCHARGE
            
    return df

In [22]:
def new_algo(df):
    
    ONE_DAY = 48
    i = df.index[0]
    while i < len(df.index):
        
        prices = df[PRICE].iloc[i:i+ONE_DAY]
        spot_price = pd.Series(prices)
        
        j = i
        while j <= ONE_DAY:
            
            if (df.at[j,PRICE] <= quantile_exc(spot_price,CHARGING_PERCENTILE)):
                df.at[j,'decision1'] = CHARGE

            if (df.at[j,PRICE] >= quantile_exc(spot_price,DISCHARGING_PERCENTILE)):
                df.at[j,'decision1'] = DISCHARGE
            j += 1
            
        i += ONE_DAY
    
    return df

In [19]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """

    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap
    
def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch

def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = math.ceil(price * dispatch * factor)
    return revenue

def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if (df.at[i,'decision1'] == CHARGE):
        raw_power = -min(POWER, (CAPACITY-opening_cap)/EFFICIENCY*2)     # CHARGE
        
    elif (df.at[i,'decision1'] == DISCHARGE):
        raw_power = min(POWER, opening_cap/EFFICIENCY*2)      # DISCHARGE
            
    return raw_power

def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df

In [20]:
def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    
    # Find Optimal Charging and Discharging period
    df3 = new_algo(df2)
    
    
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [23]:
CHARGING_PERCENTILE = 0.15
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: -28274
Total days in the dataset: 1322.0
Revenue per day: -21.38729198184569
Time Complexity for running the entire Algorithm 3: 3.2572968006134033s


In [24]:
vic_price.head(48)

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,decision1
1,2018-01-01 00:30:00,92.46,0,0,0,0,0,1
2,2018-01-01 01:00:00,87.62,0,0,0,0,0,1
3,2018-01-01 01:30:00,73.08,0,0,0,0,0,0
4,2018-01-01 02:00:00,70.18,0,0,0,0,0,0
5,2018-01-01 02:30:00,67.43,0,0,0,0,0,0
6,2018-01-01 03:00:00,66.31,0,0,0,0,0,0
7,2018-01-01 03:30:00,67.72,0,0,0,0,0,0
8,2018-01-01 04:00:00,65.50,0,0,0,0,0,0
9,2018-01-01 04:30:00,64.50,-300,-150,-9762,0,135,-1
10,2018-01-01 05:00:00,65.41,0,0,0,135,135,0


In [95]:
# Run Entire Data Pipeline (Original)
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
vic_price = run_all(vic_data)
#92550.54/day

Total revenue in the dataset: 122351814
Total days in the dataset: 1322.0
Revenue per day: 92550.54009077155
Time Complexity for running the entire Algorithm 3: 21.02471923828125s
